In [7]:
import numpy as np
import os
from pathlib import Path
import shutil
import math

In [8]:
import numpy as np
import cv2
from keras.preprocessing import image
import time
import align.detect_face
import os

import tensorflow as tf
from keras.applications.imagenet_utils import preprocess_input, decode_predictions
from keras.models import load_model
from keras.models import model_from_json
import scipy

In [9]:
emotion = "/home/topica/Downloads/extended-cohn-kanade-images/cohn-kanade-images"
labels = "/home/topica/Downloads/Emotion_labels/Emotion"
out_put = "/home/topica/workspace/data_ckp_crop"

In [10]:
list_person = os.listdir(labels)

In [11]:
# facenet 
image_size=224
margin= 44
gpu_memory_fraction=1.0

def load_and_align_data(image_path, image_size,margin, gpu_memory_fraction):
    minsize = 20 # minimum size of face
    threshold = [ 0.6, 0.7, 0.7 ]  # three steps's threshold
    factor = 0.709 # scale factor
    with tf.Graph().as_default():
        gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=gpu_memory_fraction)
        sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options, log_device_placement=False))
        with sess.as_default():
            pnet, rnet, onet = align.detect_face.create_mtcnn(sess, None)
    img = cv2.imread(os.path.expanduser(image_path))
    img_size = np.asarray(img.shape)[0:2]
    bounding_boxes, _ = align.detect_face.detect_face(img, minsize, pnet, rnet, onet, threshold, factor)
    if (len(bounding_boxes)==0):
        bb=0
        have_face = 0
    else:
        det = np.squeeze(bounding_boxes[0,0:4])
        bb = np.zeros(4, dtype=np.int32)
        bb[0] = np.maximum(det[0]-margin/2, 0)
        bb[1] = np.maximum(det[1]-margin/2, 0)
        bb[2] = np.minimum(det[2]+margin/2 - bb[0], img_size[1])
        bb[3] = np.minimum(det[3]+margin/2 - bb[1], img_size[0])
        have_face = 1
    return bb,have_face

In [ ]:
for i in range(len(list_person)):
    list_emo = os.listdir(os.path.join(labels,list_person[i]))
    for j in range(len(list_emo)):
        emo_dir = os.path.join(labels,list_person[i], list_emo[j])
        img_dir = os.path.join(emotion, list_person[i], list_emo[j])
        list_img = os.listdir(img_dir)
        list_img = sorted(list_img)
        emo_file_list = os.listdir(emo_dir)
        if len(emo_file_list)!= 0:
            emo = open(os.path.join(emo_dir,emo_file_list[0]), "r").read()
            emo = emo.split(".")[0]
            emo = str(int(emo))
            threshold = math.floor((len(list_img)-1)*0.3)
            print(threshold)
            for m in range(threshold - 1):
                image_path = os.path.join(img_dir, list_img[m])
                detect_face, have_face = load_and_align_data(image_path,image_size,margin,gpu_memory_fraction)
                if have_face != 0:        
                    detect_face = np.reshape(detect_face,(-1,4)) 
                    img = cv2.imread(image_path)
                    for (x,y,w,h) in detect_face:
                        detected_face = img[int(y):int(y+h), int(x):int(x+w)] #crop detected face
                    cv2.imwrite(os.path.join(out_put,'8',list_img[m]),detected_face)
#                 shutil.copy2(img, os.path.join(out_put,'8',list_img[m]))
            for m in range(threshold,len(list_img)):
                image_path = os.path.join(img_dir, list_img[m])
                detect_face, have_face = load_and_align_data(image_path,image_size,margin,gpu_memory_fraction)
                if have_face != 0:        
                    detect_face = np.reshape(detect_face,(-1,4)) 
                    img = cv2.imread(image_path)
                    for (x,y,w,h) in detect_face:
                        detected_face = img[int(y):int(y+h), int(x):int(x+w)] #crop detected face
                    cv2.imwrite(os.path.join(out_put,emo,list_img[m]),detected_face)
#                 shutil.copy2(img, os.path.join(out_put,emo,list_img[m]))

6
3
3
6
2
3
6
3
5
7
6
3
8
3
6
3
3
5
4
3
6
6
4
4
3
3
3
3
4
2
5
6
3
3
2
3
3
4
2
5
6
4
1
3
5
3
7
3
